# Leopard detection

In this notebook we are going to train a YOLOv5 model using the [official YOLOv5](https://github.com/ultralytics/yolov5) 🚀 notebook by Ultralytics.

[Here](https://colab.research.google.com/github/ultralytics/yolov5/blob/master/tutorial.ipynb) is the original colab notebook with the tutorial. We've used the code from there to train our leopard detection model.

# Step 1: Setup

We will first install YOLOv5 and load our drive to have access to the leopard datasets.

In [3]:
# Install dependencies
!pip install requests==2.23.0
!pip install folium==0.2.1
!pip install imgaug==0.2.5
!pip install imgaug==0.2.5
!pip install chardet==4.0.0
!pip install urllib3==1.26.6
!pip install albumentations==1.0.3
!pip install wandb
!pip install roboflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6
  Attempting uninstall: chardet
    Found existing installation: chardet 4.0.0
    Uninstalling chardet-4.0.0:
      Successfully uninstalled chardet-4.0.0
  Attempting uninstall: requests
    Found existing installation: requests 2.28.0
    Uninstalling requests-2.28.0:
      Successfully uninstalled requests-2.28.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 0.2.4 requires chardet==4.0.0, but you have chardet 3.0.4 w

In [5]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install



YOLOv5 🚀 v6.1-252-g6a67594 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.8/166.8 GB disk)


In [11]:
import torch
import utils
import os
import shutil
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-252-g6a67594 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.8/166.8 GB disk)


In [6]:
# Load the drive to access the images and annotations
from google.colab import drive
drive.mount('/content/gdrive')

# reduce the path of the drive
def driveSymboLink():
  # this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  # !ls /mydrive

driveSymboLink()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [7]:
# define the WDIR from your gdrive to the project Animal Re-Identification
WDIR = '/content/gdrive/MyDrive/MIDS/W210/Animal_Identification'

Check the Colab environment

In [8]:
from IPython.display import Image  # for displaying images

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.11.0+cu113 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


# Prepare the dataset

In [9]:
from roboflow import Roboflow
rf = Roboflow(model_format="yolov5", notebook="ultralytics")

upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


In [16]:
# set up environment
os.environ["DATASET_DIRECTORY"] = WDIR+"/datasets"

# data yaml file
dataPath=WDIR+'/yolov5/data/custom.yaml'


# Train the model

In [20]:
!python train.py --img 416 --batch 16 --epochs 150 --data $dataPath --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/gdrive/MyDrive/MIDS/W210/Animal_Identification/yolov5/data/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-252-g6a67594 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, io

In [25]:
# Save the results to the drive
shutil.copytree('/content/yolov5/runs', WDIR+'/detection/runs')

'/content/gdrive/MyDrive/MIDS/W210/Animal_Identification/detection/runs'

# Evaluate the model

In [28]:
WEIGHTS_TRAINED='/content/yolov5/runs/train/exp4/weights/best.pt'
SRC_DEV='/content/gdrive/MyDrive/MIDS/W210/Animal_Identification/datasets/leopard/dev/images'

In [29]:
!python detect.py --weights $WEIGHTS_TRAINED --img 416 --conf 0.1 --source $SRC_DEV

detect: weights=['/content/yolov5/runs/train/exp4/weights/best.pt'], source=/content/gdrive/MyDrive/MIDS/W210/Animal_Identification/datasets/leopard/dev/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-252-g6a67594 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
image 1/905 /content/gdrive/MyDrive/MIDS/W210/Animal_Identification/datasets/leopard/dev/images/giraffe_000000000063.jpg: 288x416 Done. (0.012s)
image 2/905 /content/gdrive/MyDrive/MIDS/W210/Animal_Identification/datasets/leopard/dev/images/giraffe_000000000067.jpg: 288x416 Done. 

In [30]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp2/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

Output hidden; open in https://colab.research.google.com to view.